In [6]:
import os
import pandas as pd
import toloka.client as toloka
import requests
from datetime import datetime, timedelta
import time
from tqdm import tqdm
import json

import zipfile
import glob

from multiprocessing import Pool

OAUTH_token = YOUR_TOKEN
HEADERS = {"Authorization": "OAuth %s" % OAUTH_TOKEN, "Content-Type": "application/JSON"}
toloka_client = toloka.TolokaClient(OAUTH_TOKEN, 'PRODUCTION')

URL_WORKER = 'https://toloka.dev/api/new/requester/workers/'
URL_API = "https://toloka.dev/api/v1/"

In [7]:
validator_df = pd.concat([pd.read_excel('Pora.Ai валидация.xlsx', sheet_name='Матвей Валидация Очки 1'),
                          pd.read_excel('Pora.Ai валидация.xlsx', sheet_name='Катя Валидация Макияж'),
                          pd.read_excel('Pora.Ai валидация.xlsx', sheet_name='Кристина Валидация Очки 2'),
                          pd.read_excel('Pora.Ai валидация.xlsx', sheet_name='Лиза Валидация Руки + Голов Убо'),
                          ])

In [8]:
validator_df.dropna(subset='assignment-id')

Date                         assignment-id  project   
0    2023-05-26  0002566370--646a3a5c30edd521a613054b  glasses  \
1           NaT  0002566370--646a3ac96518772244e5697d  glasses   
2           NaT  0002566370--646a3a5e461f5133df0f8abc  glasses   
3           NaT  0002566370--646a3b44539ccc5ecd37cc0f  glasses   
4           NaT  0002566370--646a3b4d539ccc5ecd37cccd  glasses   
...         ...                                   ...      ...   
3095 2023-06-03  000256938a--64711a440fbd487cc2f8979d     hand   
3096 2023-06-03  000256938a--64711cd020705c17f6a379ec     hand   
3097 2023-06-03  000256938a--64711dac0ef3d90ee8825465     hand   
3098 2023-06-03  000256938a--6471211c70a8c415f90b9760     hand   
3099 2023-06-03  000256938a--647123f32dd15e7a67f86d1c     hand   

              category        Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7   
0              glasses           glasses        NaN        NaN        NaN  \
1              glasses  glasses + makeup        NaN        NaN        NaN   
2     glasses + makeup   glasses + other        NaN        NaN        NaN   
3              glasses            makeup        NaN        NaN        NaN   
4              glasses    makeup + other        NaN        NaN        NaN   
...                ...               ...        ...        ...        ...   
3095             other               NaN        NaN        NaN        NaN   
3096             other               NaN        NaN        NaN        NaN   
3097             other               NaN        NaN        NaN        NaN   
3098             other               NaN        NaN        NaN        NaN   
3099             other               NaN        NaN        NaN        NaN   

        Not according to the instructions Unnamed: 0  Unnamed: 9  poor quality  
0     We have already received your photo        NaT         NaN           NaN  
1                      Poor photo quality        NaT         NaN           NaN  
2          Photos should not have filters        NaT         NaN           NaN  
3                                     NaN        NaT         NaN           NaN  
4                                     NaN        NaT         NaN           NaN  
...                                   ...        ...         ...           ...  
3095                                  NaN        NaT         NaN           NaN  
3096                                  NaN        NaT         NaN           NaN  
3097                                  NaN        NaT         NaN           NaN  
3098                                  NaN        NaT         NaN           NaN  
3099                                  NaN        NaT         NaN           NaN  

[6868 rows x 12 columns]

In [4]:
main_dirs_names = set()

for assignment_id in tqdm(validator_df['assignment-id'].unique()):
    main_dir_name = validator_df.loc[validator_df['assignment-id']==assignment_id, 'project'].values[0]
    main_dirs_names.add(main_dir_name)
    second_dir_name = validator_df.loc[validator_df['assignment-id']==assignment_id, 'category'].values[0]
    os.makedirs(os.path.join('photos', main_dir_name, second_dir_name), exist_ok=True)

    attachment_id = toloka_client.get_assignment(assignment_id).solutions[0].output_values['img_1']
    attachment_extension = toloka_client.get_attachment(attachment_id).name.split('.')[-1]

    full_path = os.path.join('photos', main_dir_name, second_dir_name, f'{assignment_id}.{attachment_extension}')

    if not os.path.exists(full_path):
        with open(full_path, 'wb') as file:
            toloka_client.download_attachment(attachment_id, out=file)

for main_dir in main_dirs_names:
    with zipfile.ZipFile(f'{main_dir}.zip', 'w') as f:
        for file in glob.glob(f'{main_dir}/*'):
            f.write(file)

  0%|          | 0/6862 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [14]:
def process_assignment(assignment_id):
    try:
        main_dir_name = validator_df.loc[validator_df['assignment-id']==assignment_id, 'project'].values[0]
        second_dir_name = validator_df.loc[validator_df['assignment-id']==assignment_id, 'category'].values[0]
        os.makedirs(os.path.join('photos', main_dir_name, second_dir_name), exist_ok=True)

        attachment_id = toloka_client.get_assignment(assignment_id).solutions[0].output_values['img_1']
        attachment_extension = toloka_client.get_attachment(attachment_id).name.split('.')[-1]

        full_path = os.path.join('photos', main_dir_name, second_dir_name, f'{assignment_id}.{attachment_extension}')

        if not os.path.exists(full_path):
            with open(full_path, 'wb') as file:
                toloka_client.download_attachment(attachment_id, out=file)
    except Exception as e:
        print(assignment_id, e)


main_dirs_names = set()
for assignment_id in validator_df['assignment-id'].dropna().unique():
    main_dir_name = validator_df.loc[validator_df['assignment-id']==assignment_id, 'project'].values[0]
    main_dirs_names.add(main_dir_name)

with Pool() as pool:
    pool.map(process_assignment, validator_df['assignment-id'].dropna().unique())

for main_dir in main_dirs_names:
    with zipfile.ZipFile(f'{main_dir}.zip', 'w') as f:
        for file in glob.glob(f'{main_dir}/*'):
            f.write(file)

0002566370--646a3a5c30edd521a613054b
0002566370--646a3ac96518772244e5697d
0002566370--646a3a5e461f5133df0f8abc
0002566370--646a3b44539ccc5ecd37cc0f
0002566370--646a3b4d539ccc5ecd37cccd
0002566370--646a3b37933ae74aca8a1eb5
0002566370--646a3adc6d39c7692050b165
0002566370--646a3aa833461c42f1b6f695
0002566370--646a3b3c07024f18062876f6
0002566370--646a3b32db2957659250bf19
0002566370--646a3b8c6518772244e57f6a
0002566370--646a3aad3902891188ccdf4a
0002566370--646a3b5967c7144ca92869d8
0002566370--646a3bb461c9926faffe0417
0002566370--646a3bde5fba5931d6bdf421
0002566370--646a3b6c79ae8d1f9ae45068
0002566370--646a3b7c01fd74234baaaafb
0002566370--646a3b7761c9926faffdfb09
0002566370--646a3bce30edd521a61326c9
0002566370--646a3c3af9031d2488ae9a93
0002566370--646a3cb0db2957659250ed2c
0002566370--646a3d05f9031d2488aeb669
0002566370--646a3ce71a67a94e756d740f
0002566370--646a3cf007024f1806289934
0002566370--646a3d1c59ce7910e89f8294
0002566370--646a3d0667c7144ca92890cf
0002566370--646a3d43f9031d2488aebd55
0

Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_assignment' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framew

KeyboardInterrupt: 